In [1]:
import cv2
import numpy as np
import os

In [2]:
#knn algorithm for prediction
def distance(p1,p2):
    return (np.sum((p2-p1)**2))**0.5




def knn(X,Y,test,k=5):
    m=X.shape[0]
    d=[]
    for i in range(m):
        dist = distance(X[i],test)
        d.append((dist,Y[i]))
    d = np.array(sorted(d))[:,1]
    d = d[:k]
    t = np.unique(d,return_counts=True)
    idx = np.argmax(t[1])
    pred = int(t[0][idx])
    return pred

In [3]:
#data preparation
dataset_path = "./Datasets/face_data/"
names = {}
class_id = 0
face_data = []
labels = []

for fx in os.listdir(dataset_path):
    if fx.endswith(".npy"):
        names[class_id] = fx[:-4]
        print("loading file",fx)
        data_item = np.load(dataset_path + fx)
        face_data.append(data_item)
        
        target = class_id*np.ones((data_item.shape[0],))
        labels.append(target)
        class_id+=1

X = np.concatenate(face_data,axis=0)
Y = np.concatenate(labels,axis=0)

loading file abhishek.npy


In [4]:
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

while True:
    ret,frame = cap.read()
    if ret == False:
        continue
        
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    if len(faces) == 0:
        continue
        
    for (x,y,w,h) in faces:
        offset = 10
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))
        
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),0)
        
    test = face_section.flatten()
    pred = knn(X,Y,test)
    
    pred = names[pred]
    
    cv2.putText(frame,pred, (x,y), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
    cv2.imshow('VideoCapture',frame)
    
    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

        
        
    